In [1]:
import json
import pandas as pd
import ast
from typing import Dict
from datetime import datetime

In [2]:
import pandas as pd
import json
from typing import Dict
from datetime import datetime

# Ruta a tu archivo JSON
archivo = 'steam_games.json/output_steam_games.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(json.loads(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Asegúrate de que 'release_date' sea un objeto datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', format='%Y-%m-%d')

# Calcular las horas desde el lanzamiento hasta la fecha actual
now = datetime.now()
df['hours_since_release'] = (now - df['release_date']).dt.total_seconds() / 3600

def PlayTimeGenre(genero: str) -> Dict[str, int]:
    # Asegúrate de que todos los valores en 'genres' sean listas
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    
    # Filtrar el DataFrame por el género especificado
    df_genre = df[df['genres'].apply(lambda x: genero in x)]
    
    # Agrupar por año de lanzamiento y sumar las horas desde el lanzamiento
    df_genre_year = df_genre.groupby(df_genre['release_date'].dt.year)['hours_since_release'].sum()
    
    # Encontrar el año de lanzamiento con más horas desde el lanzamiento
    max_year = df_genre_year.idxmax()
    
    return {f"Año de lanzamiento con más horas jugadas para {genero}" : max_year}



resultado = PlayTimeGenre("Casual")
print(resultado)


{'Año de lanzamiento con más horas jugadas para Casual': 2017.0}


In [3]:
#Para practicamente todos los Generos el maximo de horas jugadas es para el año 2017 ya que este año es 
#El año de lanzamiento de la más lanzamientos 9518

In [4]:
# Contar cuántos juegos se lanzaron cada año
df['release_year'] = df['release_date'].dt.year
games_per_year = df['release_year'].value_counts().sort_index()

print(games_per_year)


release_year
1970.0       2
1975.0       1
1980.0       1
1981.0       3
1982.0       3
1983.0       4
1984.0       4
1985.0       3
1986.0       1
1987.0       9
1988.0       7
1989.0      10
1990.0      12
1991.0      16
1992.0      15
1993.0      34
1994.0      36
1995.0      33
1996.0      39
1997.0      42
1998.0      61
1999.0      55
2000.0      40
2001.0      69
2002.0      50
2003.0      84
2004.0      79
2005.0     102
2006.0     155
2007.0     167
2008.0     213
2009.0     337
2010.0     436
2011.0     589
2012.0    1192
2013.0    1551
2014.0    2832
2015.0    4945
2016.0    6933
2017.0    9518
2018.0      96
2019.0       3
2021.0       1
Name: count, dtype: int64


In [5]:
#API

In [6]:
import nest_asyncio
from fastapi import FastAPI
from starlette.testclient import TestClient
from typing import Dict
import pandas as pd
import json
from datetime import datetime

# Esto permite a Jupyter ejecutar código asíncrono
nest_asyncio.apply()

# Ruta a tu archivo JSON
archivo = 'output_steam_games.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(json.loads(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Asegúrate de que 'release_date' sea un objeto datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', format='%Y-%m-%d')

# Calcular las horas desde el lanzamiento hasta la fecha actual
now = datetime.now()
df['hours_since_release'] = (now - df['release_date']).dt.total_seconds() / 3600

app = FastAPI()

@app.get("/PlayTimeGenre/{genero}")
def PlayTimeGenre(genero: str) -> Dict[str, int]:
    # Asegúrate de que todos los valores en 'genres' sean listas
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    
    # Filtrar el DataFrame por el género especificado
    df_genre = df[df['genres'].apply(lambda x: genero in x)]
    
    # Agrupar por año de lanzamiento y sumar las horas desde el lanzamiento
    df_genre_year = df_genre.groupby(df_genre['release_date'].dt.year)['hours_since_release'].sum()
    
    # Encontrar el año de lanzamiento con más horas desde el lanzamiento
    max_year = df_genre_year.idxmax()
    
    return {f"Año de lanzamiento con más horas jugadas para {genero}" : max_year}

# Crear un cliente de prueba
client = TestClient(app)

# Ahora puedes hacer solicitudes a tu API
response = client.get("/PlayTimeGenre/Action")
print(response.json())


{'Año de lanzamiento con más horas jugadas para Action': 2017}


In [7]:
#ahora vamos con la 2da funcion

In [8]:
#Tomamos australian_user_reviews.json --> 'user_id': '76561197970982479' y 'reviews' --> 'item_id': '22200' y 'posted'
#Asumiremos que el usuario ha jugado desde que comento la reseña hasta la fecha de hoy

In [9]:
import ast
import pandas as pd
import datetime
import re

# Ruta a tu archivo JSON
archivo = 'user_reviews.json/australian_user_reviews.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(ast.literal_eval(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Lista para guardar las filas del nuevo DataFrame
filas = []

# Extraer solo el campo 'recommend' de cada revisión
for index, row in df.iterrows():
    user_id = row['user_id']
    for review in row['reviews']:
        item_id = review['item_id']
        posted = review['posted']
        
        # Extraer la fecha del string 'posted' usando una expresión regular
        date_match = re.search(r'Posted ([A-Za-z]+ \d+(, \d+)?)', posted)
        if date_match:
            date_str = date_match.group(1)
            # Intentar convertir la fecha en un objeto datetime
            try:
                posted_date = datetime.datetime.strptime(date_str, '%B %d, %Y')
            except ValueError:
                try:
                    # Si no se puede convertir con el año, intentar sin el año
                    posted_date = datetime.datetime.strptime(date_str, '%B %d')
                except ValueError:
                    # Si el día está fuera de rango para el mes, continuar con la siguiente fila
                    continue
        
        # Añadir la fila a la lista de filas
        filas.append({'user_id': user_id, 'item_id': item_id, 'posted_date': posted_date})

# Crear el DataFrame a partir de la lista de filas
df_userid_item_id1 = pd.DataFrame(filas)

# Imprimir el DataFrame
print(df_userid_item_id1)




                 user_id item_id posted_date
0      76561197970982479    1250  2011-11-05
1      76561197970982479   22200  2011-07-15
2      76561197970982479   43110  2011-04-21
3                js41637  251610  2014-06-24
4                js41637  227300  2013-09-08
...                  ...     ...         ...
59275  76561198312638244      70  1900-07-10
59276  76561198312638244  362890  1900-07-08
59277        LydiaMorley  273110  1900-07-03
59278        LydiaMorley     730  1900-07-20
59279        LydiaMorley     440  1900-07-02

[59280 rows x 3 columns]


In [24]:
#Tomamos australian_user_item.json --> 'user_id' y 'items' --> 'item_id': '10' , 'item_name': 'Counter-Strike' y 'playtime_forever' con valor en horas

In [11]:
###**** Optimizar codigo, esta muy lento o trabajar en CSV para mejor rendimiento

import ast
import pandas as pd

# Ruta a tu archivo JSON
archivo = 'users_items.json/australian_users_items.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(ast.literal_eval(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Lista para guardar las filas del nuevo DataFrame
filas = []

# Iterar sobre cada fila del DataFrame original
for index, row in df.iterrows():
    user_id = row['user_id']
    items = row['items']
    
    # Iterar sobre la lista de items en cada fila
    for item in items:
        item_id = item['item_id']
        playtime_forever = item['playtime_forever']
        
        # Añadir la fila a la lista de filas
        filas.append({'user_id': user_id, 'item_id': item_id, 'playtime_forever': playtime_forever})

# Crear el DataFrame a partir de la lista de filas
df_userid_item_id_playtime = pd.DataFrame(filas)

# Imprimir el DataFrame resultante
print(df_userid_item_id_playtime)



                   user_id item_id  playtime_forever
0        76561197970982479      10                 6
1        76561197970982479      20                 0
2        76561197970982479      30                 7
3        76561197970982479      40                 0
4        76561197970982479      50                 0
...                    ...     ...               ...
5153204  76561198329548331  346330                 0
5153205  76561198329548331  373330                 0
5153206  76561198329548331  388490                 3
5153207  76561198329548331  521570                 4
5153208  76561198329548331  519140                 3

[5153209 rows x 3 columns]


In [73]:
import ast
import pandas as pd

# Ruta a tu archivo JSON
archivo = 'users_items.json/australian_users_items.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(ast.literal_eval(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Lista para guardar las filas del nuevo DataFrame
filas = []

# Iterar sobre cada fila del DataFrame original
for index, row in df.iterrows():
    user_id = row['user_id']
    items = row['items']
    
    # Iterar sobre la lista de items en cada fila
    for item in items:
        item_id = item['item_id']
        item_name = item['item_name']  # Añadir item_name
        playtime_forever = item['playtime_forever']
        
        # Añadir la fila a la lista de filas
        filas.append({'user_id': user_id, 'item_id': item_id, 'item_name': item_name, 'playtime_forever': playtime_forever})  # Añadir item_name al DataFrame

# Crear el DataFrame a partir de la lista de filas
df_userid_item_id_playtime = pd.DataFrame(filas)

# Imprimir el DataFrame resultante
print(df_userid_item_id_playtime)


                   user_id item_id                      item_name  \
0        76561197970982479      10                 Counter-Strike   
1        76561197970982479      20          Team Fortress Classic   
2        76561197970982479      30                  Day of Defeat   
3        76561197970982479      40             Deathmatch Classic   
4        76561197970982479      50      Half-Life: Opposing Force   
...                    ...     ...                            ...   
5153204  76561198329548331  346330                   BrainBread 2   
5153205  76561198329548331  373330                    All Is Dust   
5153206  76561198329548331  388490  One Way To Die: Steam Edition   
5153207  76561198329548331  521570          You Have 10 Seconds 2   
5153208  76561198329548331  519140                     Minds Eyes   

         playtime_forever  
0                       6  
1                       0  
2                       7  
3                       0  
4                       0  
...

In [75]:
# Eliminar las filas con valores NaN en al menos una de las columnas 'user_id', 'item_id' y 'playtime_forever'
df_userid_item_id_playtime = df_userid_item_id_playtime.dropna(subset=['user_id', 'item_id', 'item_name', 'playtime_forever'], how='any')

# Imprimir el DataFrame resultante
print(df_userid_item_id_playtime)


                   user_id item_id                      item_name  \
0        76561197970982479      10                 Counter-Strike   
1        76561197970982479      20          Team Fortress Classic   
2        76561197970982479      30                  Day of Defeat   
3        76561197970982479      40             Deathmatch Classic   
4        76561197970982479      50      Half-Life: Opposing Force   
...                    ...     ...                            ...   
5153204  76561198329548331  346330                   BrainBread 2   
5153205  76561198329548331  373330                    All Is Dust   
5153206  76561198329548331  388490  One Way To Die: Steam Edition   
5153207  76561198329548331  521570          You Have 10 Seconds 2   
5153208  76561198329548331  519140                     Minds Eyes   

         playtime_forever  
0                       6  
1                       0  
2                       7  
3                       0  
4                       0  
...

In [64]:
import json
import pandas as pd

# Ruta a tu archivo JSON
archivo = 'output_steam_games.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(json.loads(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Lista para guardar las filas del nuevo DataFrame
filas = []

# Iterar sobre cada fila del DataFrame original
for index, row in df.iterrows():
    genres = row['genres']
    title = row['title']
        
    # Añadir la fila a la lista de filas
    filas.append({'genres': genres, 'title': title})

# Crear el DataFrame a partir de la lista de filas
df_genres_title = pd.DataFrame(filas)

# Eliminar las filas con valores NaN en ambas columnas 'genres' y 'title'
df_genres_title = df_genres_title.dropna(subset=['genres', 'title'], how='all')


# Imprimir el DataFrame resultante
print(df_genres_title)


                                                   genres  \
88310       [Action, Casual, Indie, Simulation, Strategy]   
88311                [Free to Play, Indie, RPG, Strategy]   
88312   [Casual, Free to Play, Indie, Simulation, Sports]   
88313                         [Action, Adventure, Casual]   
88315                     [Action, Adventure, Simulation]   
...                                                   ...   
120439                 [Action, Adventure, Casual, Indie]   
120440              [Casual, Indie, Simulation, Strategy]   
120441                          [Casual, Indie, Strategy]   
120442                        [Indie, Racing, Simulation]   
120443                                    [Casual, Indie]   

                           title  
88310        Lost Summoner Kitty  
88311                  Ironbound  
88312    Real Pool 3D - Poolians  
88313                    弹炸人2222  
88315      Battle Royale Trainer  
...                          ...  
120439              Ke

In [13]:
#Union de los 2 datasets

In [76]:
# Unir los DataFrames en las columnas 'user_id' y 'item_id'
df_merged_df_userid_item_id_playtime_df_userid_item_id1 = pd.merge(df_userid_item_id_playtime, df_userid_item_id1, on=['user_id', 'item_id'], how='inner')

# Imprimir el DataFrame resultante
print(df_merged_df_userid_item_id_playtime_df_userid_item_id1)


                 user_id item_id                         item_name  \
0      76561197970982479   22200                        Zeno Clash   
1      76561197970982479    1250                     Killing Floor   
2      76561197970982479   43110                        Metro 2033   
3                js41637  227300            Euro Truck Simulator 2   
4                js41637  239030                    Papers, Please   
...                  ...     ...                               ...   
46291           Ghoustik     730  Counter-Strike: Global Offensive   
46292  76561198312638244      70                         Half-Life   
46293  76561198312638244     130             Half-Life: Blue Shift   
46294  76561198312638244  233270           Far Cry® 3 Blood Dragon   
46295  76561198312638244  362890                        Black Mesa   

       playtime_forever posted_date  
0                   271  2011-07-15  
1                 10006  2011-11-05  
2                   834  2011-04-21  
3      

In [77]:
# Convertir la columna 'posted_date' a datetime
df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'] = pd.to_datetime(df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'])

# Crear un filtro para las fechas anteriores a 1990
filtro = df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'].dt.year < 1990

# Contar el número de filas con fechas anteriores a 1990
num_filas = df_merged_df_userid_item_id_playtime_df_userid_item_id1[filtro].shape[0]

print(f'Hay {num_filas} filas con fechas anteriores a 1990.')


Hay 8347 filas con fechas anteriores a 1990.


In [79]:
# Crear un filtro para las fechas posteriores a 1989
filtro = df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'].dt.year >= 1990

# Aplicar el filtro al DataFrame
df_merged = df_merged_df_userid_item_id_playtime_df_userid_item_id1[filtro]

# Imprimir el DataFrame resultante
print(df_merged)


                 user_id item_id  \
0      76561197970982479   22200   
1      76561197970982479    1250   
2      76561197970982479   43110   
3                js41637  227300   
4                js41637  239030   
...                  ...     ...   
46261            wayfeng     730   
46263  76561198251004808  253980   
46268        72947282842     730   
46269          ApxLGhost     730   
46276  76561198267374962  369200   

                                               item_name  playtime_forever  \
0                                             Zeno Clash               271   
1                                          Killing Floor             10006   
2                                             Metro 2033               834   
3                                 Euro Truck Simulator 2               551   
4                                         Papers, Please               349   
...                                                  ...               ...   
46261                

In [ ]:
#Union con el DF que falta (genres - title)

In [83]:
# Unir los DataFrames en las columnas 'item_name' y 'title'
df_merged1 = pd.merge(df_merged, df_genres_title, left_on='item_name', right_on='title', how='inner')

# Imprimir el DataFrame resultante
print(df_merged1)


                         user_id item_id            item_name  \
0              76561197970982479   22200           Zeno Clash   
1      seantheextraprawnsheepguy   22200           Zeno Clash   
2                     pipekissXD   22200           Zeno Clash   
3               ClockworkLunatic   22200           Zeno Clash   
4                      vschiffer   22200           Zeno Clash   
...                          ...     ...                  ...   
27221                    K1NGCJS  294230               Millie   
27222             laislabonita75  305920  Another Perspective   
27223             laislabonita75  367780         Aero's Quest   
27224            evilindiegaming  367780         Aero's Quest   
27225                llDracuwulf  307130              Asteria   

       playtime_forever posted_date                      genres  \
0                   271  2011-07-15             [Action, Indie]   
1                   240  2011-07-16             [Action, Indie]   
2                 

In [90]:
def UserForGenre(df_merged1, genero):
    # Filtrar el DataFrame por género
    df_genre = df_merged1[df_merged1['genres'].apply(lambda x: genero in x if isinstance(x, list) else False)]
    
    # Agrupar por 'user_id' y sumar 'playtime_forever'
    df_grouped = df_genre.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Encontrar el usuario con más horas jugadas
    max_playtime_user = df_grouped[df_grouped['playtime_forever'] == df_grouped['playtime_forever'].max()]['user_id'].values[0]
    
    # Crear un DataFrame con las horas jugadas por año
    df_genre['year'] = df_genre['posted_date'].dt.year
    df_hours_per_year = df_genre.groupby('year')['playtime_forever'].sum().reset_index()
    hours_per_year = df_hours_per_year.to_dict('records')
    
    return {"Usuario con más horas jugadas para Género X": max_playtime_user, "Horas jugadas": hours_per_year}

# Prueba la función con el género "Action"
resultado = UserForGenre(df_merged1, "Simulation")

# Imprime el resultado
print(resultado)


{'Usuario con más horas jugadas para Género X': '76561198063648921', 'Horas jugadas': [{'year': 2010, 'playtime_forever': 50721}, {'year': 2011, 'playtime_forever': 1408086}, {'year': 2012, 'playtime_forever': 1660938}, {'year': 2013, 'playtime_forever': 11241102}, {'year': 2014, 'playtime_forever': 28414232}, {'year': 2015, 'playtime_forever': 24196292}]}


C:\Users\MW6529\AppData\Local\Temp\ipykernel_19476\2084833676.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre['year'] = df_genre['posted_date'].dt.year
